# Allocation Algorithm 7

> **1.** Create a list of dataframes.  Each dataframe consists of an EI and all the children assigned to that EI in the real data

> **2.** Create a cluster center with is an average of these values for each dataframe

> **3.** Assign children to clusters (Children are in order based on their distance to their closest EI in descending order).  After an EI has 17 children, no more children are assigned to them to create somewhat even distribution

> **4.** Push allocation to MySQL table locationdataalgo6

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("Select ei_id, longitude, latitude from ei "
         "WHERE ei_id != '65d75e28e09e67e025e8cd3c'")
cursor.execute(query)

ei_ids = cursor.fetchall()

eiDFs = []

for ei in ei_ids:
    ei_group_list = []
    data = {}
    data["id"] = ei[0]
    data["longitude"] = ei[1]
    data["latitude"] = ei[2]
    ei_child_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE ei_id_1 = '{ei[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(ei_child_query)
    ei_child = cursor.fetchall()
    
    for row2 in ei_child:
        # Add columns with prefixes
        data[f'{row2[0]}_distance'] = row2[2]
        data[f'{row2[0]}_time'] = row2[3]
        
    ei1_ei_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_1 = '{ei[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei1_ei_query)
    ei1_ei = cursor.fetchall()
    
    for row3 in ei1_ei:
        data[f'{row3[1]}_distance'] = row3[2]
        data[f'{row3[1]}_time'] = row3[3]
    
    ei_ei2_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_2 = '{ei[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei_ei2_query)
    ei_ei2 = cursor.fetchall()
    
    for row4 in ei_ei2:
        data[f'{row4[0]}_distance'] = row4[2]
        data[f'{row4[0]}_time'] = row4[3]
        
    ei_group_list.append(data)
    
    childQuery = ("SELECT child.child_id, child.longitude, child.latitude FROM locationdatareal ld "
                  "join child on child.child_id = ld.child_id_1 "
                  f"where child_id_2 is null and ei_id_2 is null and ld.assigned = 1 and ei_id_1 = '{ei[0]}'")
    
    cursor.execute(childQuery)
    assignedChild = cursor.fetchall()
    
    for child in assignedChild:
        data = {}
        data["id"] = child[0]
        data["longitude"] = child[1]
        data["latitude"] = child[2]
        child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              "FROM locationdata "
                              f"WHERE child_id_1 = '{child[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child1_child_query)
        child1_child = cursor.fetchall()

        for row2 in child1_child:
            # Add columns with prefixes
            data[f'{row2[1]}_distance'] = row2[2]
            data[f'{row2[1]}_time'] = row2[3]

        child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              "FROM locationdata "
                              f"WHERE child_id_2 = '{child[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_child2_query)
        child_child2 = cursor.fetchall()

        for row3 in child_child2:
            data[f'{row3[0]}_distance'] = row3[2]
            data[f'{row3[0]}_time'] = row3[3]

        child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{child[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_ei_query)
        child_ei = cursor.fetchall()

        for row4 in child_ei:
            data[f'{row4[1]}_distance'] = row4[2]
            data[f'{row4[1]}_time'] = row4[3]

        ei_group_list.append(data)
    eiDataFrame = pd.DataFrame(ei_group_list)
    eiDataFrame = eiDataFrame.fillna(0)
    eiDFs.append(eiDataFrame)

In [3]:
clusterCenters = pd.DataFrame()
for df in eiDFs:
    id_value = df.loc[0, "id"]
    df = df.drop(columns=["id"])
    #print(df)
    new_row = df.mean()
    new_row["id"] = id_value
    new_row["assignment_count"] = 0
    new_row = new_row.to_frame()
    new_row = new_row.transpose()
    clusterCenters = pd.concat([clusterCenters, new_row], ignore_index = True)

print(clusterCenters)

    longitude   latitude 65d8d05fe09e67e025e8cda9_distance  \
0  -82.379945  36.347119                            26.851   
1  -82.092944  36.536099                          55.14825   
2  -82.970359  36.278992                          29.13344   
3   -82.47012   36.30504                         20.892373   
4  -81.876901  36.457192                           61.6301   
5  -82.572813  36.533388                         45.247069   
6  -82.822728  36.467722                         43.196948   
7  -82.605034  36.252508                         14.238766   
8  -82.219225  36.557028                         49.248005   
9   -82.27384  36.344158                           31.4583   
10 -82.197074  36.357603                         40.100368   
11 -82.550778  36.526778                         39.698178   

   65d8d05fe09e67e025e8cda9_time 65d8d732e09e67e025e8cdb6_distance  \
0                      37.415495                         26.511084   
1                      69.944883                     

In [4]:
query = ("Select child_id_1, child.longitude, child.latitude, min(driving_distance) shortest_distance "
         "from locationdata ld "
         "join child on child.child_id = ld.child_id_1 "
         "where child_id_2 is NULL and ei_id_2 is NULL "
         "group by child_id_1 "
         "order by shortest_distance desc")
cursor.execute(query)

child_ids = cursor.fetchall()

data_list = []
for row in child_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child1_child_query)
    child1_child = cursor.fetchall()
    
    for row2 in child1_child:
        # Add columns with prefixes
        data[f'{row2[1]}_distance'] = row2[2]
        data[f'{row2[1]}_time'] = row2[3]
        
    child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_child2_query)
    child_child2 = cursor.fetchall()
    
    for row3 in child_child2:
        data[f'{row3[0]}_distance'] = row3[2]
        data[f'{row3[0]}_time'] = row3[3]
    
    child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_ei_query)
    child_ei = cursor.fetchall()
    
    for row4 in child_ei:
        data[f'{row4[1]}_distance'] = row4[2]
        data[f'{row4[1]}_time'] = row4[3]
        
    data_list.append(data)
        
childDataFrame = pd.DataFrame(data_list)
childDataFrame = childDataFrame.fillna(0)
childDataFrame["assigned"] = ""

print(childDataFrame)

                           id  longitude   latitude  \
0    65ef076606e0a866af56d202 -81.712128  36.476418   
1    65d787f5e09e67e025e8cd71 -81.738327  36.436672   
2    65d7886ae09e67e025e8cd72 -81.738327  36.436672   
3    65d76d38e09e67e025e8cd4e -83.244423  36.348007   
4    65e5d9ad38f826f292672007 -81.751564  36.453678   
..                        ...        ...        ...   
191  65e5d71c38f826f292672001 -82.373520  36.409969   
192  65d8d6d9e09e67e025e8cdb5 -82.212234  36.589123   
193  65e5d4d238f826f292671ffc -82.374298  36.409046   
194  65e5d57838f826f292671ffd -82.374298  36.409046   
195  65d8da66e09e67e025e8cdbd -82.457680  36.297832   

     65d76430e09e67e025e8cd43_distance  65d76430e09e67e025e8cd43_time  \
0                             38.87500                        68.4574   
1                             37.45180                        65.5944   
2                             37.45180                        65.5944   
3                             76.31480         

In [5]:
childDataFrameOrdered = childDataFrame.sort_index(axis=1)
clusterCentersOrdered = clusterCenters.sort_index(axis=1)

In [6]:
def euclidean_distance(childRow, eiRow):
    child_values = childRow.drop(['id', 'assigned']).values
    ei_values = eiRow.drop(['id', 'assignment_count']).values
    return np.sqrt(np.sum((child_values - ei_values) ** 2))

In [7]:
childDataFrameAllocate = childDataFrameOrdered.copy()
for child_index, child_row in childDataFrameOrdered.iterrows():
    if childDataFrameAllocate.loc[child_index, "assigned"] == "":
        availableClusters = clusterCentersOrdered[clusterCentersOrdered['assignment_count'] < 17]
        eiDF = pd.DataFrame(columns=["id", "euclidean_distance"])
        for cluster_index, cluster_row in availableClusters.iterrows():
            e_value = euclidean_distance(child_row, cluster_row)
            eiDF = pd.concat([eiDF, pd.DataFrame({"id": [cluster_row["id"]], "euclidean_distance": [e_value]})], ignore_index=True)
        minValue = eiDF["euclidean_distance"].min()
        minIndex = eiDF["euclidean_distance"].idxmin()
        ei_with_min = eiDF.at[minIndex, "id"]

        same_location_children = childDataFrameOrdered[(childDataFrameOrdered["longitude"] == child_row["longitude"]) & 
                                                      (childDataFrameOrdered["latitude"] == child_row["latitude"])]
        childDataFrameOrdered.loc[same_location_children.index, 'assigned'] = ei_with_min
        
        rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_with_min]
        count_ei_with_min = len(rows_with_ei_with_min)
        
        clusters_index = clusterCentersOrdered[clusterCentersOrdered['id'] == ei_with_min].index
        clusterCentersOrdered.at[clusters_index[0], 'assignment_count'] = count_ei_with_min
    else:
        print("child already assigned")
    childDataFrameAllocate = childDataFrameOrdered.copy()

print(clusterCentersOrdered)

child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
   65d75a2ce09e67e025e8cd30_distance 65d75a2ce09e67e025e8cd30_time  \
0                           5.621147                     11.083366   
1                          31.606157                       41.7229   
2                            44.1713                      54.59888   
3                           9.180453                     15.768588   
4                           40.04035                     60.475833   
5                           23.47793                     28.994825   
6                          39.430983              

In [8]:
for index, cluster in clusterCentersOrdered.iterrows():
    cluster_id = cluster["id"]
    rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == cluster_id]
    count_ei_with_min = len(rows_with_ei_with_min)
    print(f"children assigned to this ei: {count_ei_with_min}")

children assigned to this ei: 17
children assigned to this ei: 9
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 18
children assigned to this ei: 14
children assigned to this ei: 17
children assigned to this ei: 18
children assigned to this ei: 17
children assigned to this ei: 18
children assigned to this ei: 17


In [9]:
for index, child in childDataFrameOrdered.iterrows():
    child_id = child["id"]
    ei_id = child["assigned"]
    updateQuery = ("Update locationdataalgo7 "
                   "SET assigned = 1 "
                   f"WHERE child_id_1 = '{child_id}' and ei_id_1 = '{ei_id}' and child_id_2 is NULL and ei_id_2 is NULL")
    cursor.execute(updateQuery)
    connection.commit()
    print("Update query pushed")

Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
